## 제1유형

### ✏️ mcars 데이터셋의 qsec 컬럼을 최소-최대 척도(Min-Max Scale)로 변환한 후 0.5 보다 큰 값을 갖는 레코드 수를 구하시오.

In [43]:
# 데이터 불러오기
import pandas as pd
df = pd.read_csv('./data/mcars.csv', index_col=0)
print(df.head())

                    mpg  cyl   disp   hp  drat   wt  qsec  vs  am  gear  carb
Unnamed: 0                                                                   
Mazda RX4         21.00    6 160.00  110  3.90 2.62 16.46   0   1     4     4
Mazda RX4 Wag     21.00    6 160.00  110  3.90 2.88 17.02   0   1     4     4
Datsun 710        22.80    4 108.00   93  3.85 2.32 18.61   1   1     4     1
Hornet 4 Drive    21.40    6 258.00  110  3.08 3.21 19.44   1   0     3     1
Hornet Sportabout 18.70    8 360.00  175  3.15 3.44 17.02   0   0     3     2


- qsec 컬럼에 min-max 스케일로 변환하고, 0.5 보다 큰 값 찾기

- 사이킷런의 MinMaxScaler 사용시 0과 1 사이로 변환 가능

- MinMaxScaler() 를 scaler 변수에 담고 scaler.fit_transform(df[['qsec']]) t사용해 qsec 컬럼 값 변환

    - fit_transform() 함수는 데이터프레임 형태로 입력해야 함

        - df[['qsec']] 처럼 대괄호 두 번 사용해 데이터프레임 형태로 만듦

- 1개 컬럼 선택 시 데이터프레임과 시리즈 자료형

    - df[['qsec']] : 데이터프레임

    - df['qsec'] : 시리즈

In [44]:
# MinMaxScaler 활용
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
print('MinMax Scale 변환 전 :\n', df['qsec'].head())
df['qsec'] = scaler.fit_transform(df[['qsec']])
print('\nMinMax Scale 변환 후 :\n', df['qsec'].head())

MinMax Scale 변환 전 :
 Unnamed: 0
Mazda RX4           16.46
Mazda RX4 Wag       17.02
Datsun 710          18.61
Hornet 4 Drive      19.44
Hornet Sportabout   17.02
Name: qsec, dtype: float64

MinMax Scale 변환 후 :
 Unnamed: 0
Mazda RX4           0.23
Mazda RX4 Wag       0.30
Datsun 710          0.49
Hornet 4 Drive      0.59
Hornet Sportabout   0.30
Name: qsec, dtype: float64


In [45]:
# sum 활용
cond = df['qsec'] > 0.5     # True(1), False(0) 로 반환됨
print(sum(cond))            # True 인 데이터의 개수(조건을 만족하는 데이터의 개수)

9


In [46]:
# len 활용
cond = df['qsec'] > 0.5     # 데이터프레임 필터링
print(len(df[cond]))        # len() : 데이터프레임에 있는 행(레코드) 수 반환

9


## 제 2유형

### ✏️ 제공된 학습용 데이터(train)을 이용하여 백화점 구매 고객의 성별을 예측하는 모델을 개발하고, 개발한 모델에 기반하여 평가용 데이터(test)에 적용해 얻은 성별 예측 결과를 [제출 형식]에 따라 csv 파일로 생성하여 제출하시오. (예측 결과는 ROC-AUC 평가지표에 따라 평가함)

In [47]:
# 라이브러리 및 데이터 불러오기
import pandas as pd

train = pd.read_csv('https://dataq.goorm.io/input-file/content?quizIndex=q_iIhBr_1698648603412&fileName=data%2Fcustomer_train.csv')
test = pd.read_csv('https://dataq.goorm.io/input-file/content?quizIndex=q_iIhBr_1698648603412&fileName=data%2Fcustomer_test.csv')

# 데이터 샘플 확인 : 카테고리(문자)와 연속형(숫자) 컬럼 혼합
## 컬럼 다 보고싶으면 pd.set_option('display.max_columns', None)
print(train.head())
print(test.head())

# 탐색적 데이터 분석(EDA) : 데이터 크기 확인, train 은 target 컬럼 포함해 test 보다 컬럼 수가 1개 많음
print(train.shape, test.shape)

   회원ID      총구매액     최대구매액       환불금액   주구매상품 주구매지점  방문일수  방문당구매건수  주말방문비율  \
0     0  68282840  11264000 6860000.00      기타   강남점    19     3.89    0.53   
1     1   2136000   2136000  300000.00     스포츠   잠실점     2     1.50    0.00   
2     2   3197000   1639000        NaN  남성 캐주얼   관악점     2     2.00    0.00   
3     3  16077620   4935000        NaN      기타   광주점    18     2.44    0.32   
4     4  29050000  24000000        NaN      보석  본  점     2     1.50    0.00   

   구매주기  성별  
0    17   0  
1     1   0  
2     1   1  
3    16   1  
4    85   0  
   회원ID       총구매액     최대구매액        환불금액 주구매상품 주구매지점  방문일수  방문당구매건수  주말방문비율  \
0  3500   70900400  22000000  4050000.00    골프  부산본점    13     1.46    0.79   
1  3501  310533100  38558000 48034700.00   농산물   잠실점    90     2.43    0.37   
2  3502  305264140  14825000 30521000.00  가공식품  본  점   101    14.62    0.08   
3  3503    7594080   5225000         NaN  주방용품  부산본점     5     2.00    0.00   
4  3504    1795790   1411200         NaN   수산품

- 평가 기준 : ROC-AUC 평가

    - 실제 양성 클래스를 양성으로 잘 예측하는 능력(진짜 양성 비율)과 음성 클래스를 잘못 양성으로 예측하는 능력(거짓 양성 비율) 간의 관계를 평가해 모델의 분류 성능을 나타내는 지표

- label(target) 성별 : 0(여자), 1(남자)

    - '남성 확률값을 예측하라'는 구체적인 지시는 없지만, 이진 분류에서는 일반적으로 1로 표기된 클래스가 양성(positive)

- 제출 방식은 test 데이터로 성별 예측 결과 제출 : 남자일 확률을 csv 로 제출 (파일명 : result.csv)

In [48]:
# 데이터 자료형(타입) 확인 : float 3개, int64 6개, object 2개
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   회원ID     3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   방문일수     3500 non-null   int64  
 7   방문당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
 10  성별       3500 non-null   int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 300.9+ KB
None


In [49]:
# 기초 통계값 : 일부 숫자가 지수 표기법으로 표기
print(train.describe())

         회원ID          총구매액        최대구매액         환불금액    방문일수  방문당구매건수  \
count 3500.00       3500.00      3500.00      1205.00 3500.00  3500.00   
mean  1749.50   91919251.80  19664241.78  24078215.99   19.25     2.83   
std   1010.51  163506530.31  31992347.94  47464527.82   27.17     1.91   
min      0.00  -52421520.00  -2992000.00      5600.00    1.00     1.00   
25%    874.75    4747050.00   2875000.00   2259000.00    2.00     1.67   
50%   1749.50   28222700.00   9837000.00   7392000.00    8.00     2.33   
75%   2624.25  106507930.00  22962500.00  24120000.00   25.00     3.38   
max   3499.00 2323180070.00 706629000.00 563753000.00  285.00    22.08   

       주말방문비율    구매주기      성별  
count 3500.00 3500.00 3500.00  
mean     0.31   20.96    0.38  
std      0.29   24.75    0.48  
min      0.00    0.00    0.00  
25%      0.03    4.00    0.00  
50%      0.26   13.00    0.00  
75%      0.45   28.00    1.00  
max      1.00  166.00    1.00  


In [50]:
# 판다스 설정 변경하면 수치를 일반 표기법으로 표시 가능
pd.set_option('display.float_format', '{:.2f}'.format)
print(train.describe())

         회원ID          총구매액        최대구매액         환불금액    방문일수  방문당구매건수  \
count 3500.00       3500.00      3500.00      1205.00 3500.00  3500.00   
mean  1749.50   91919251.80  19664241.78  24078215.99   19.25     2.83   
std   1010.51  163506530.31  31992347.94  47464527.82   27.17     1.91   
min      0.00  -52421520.00  -2992000.00      5600.00    1.00     1.00   
25%    874.75    4747050.00   2875000.00   2259000.00    2.00     1.67   
50%   1749.50   28222700.00   9837000.00   7392000.00    8.00     2.33   
75%   2624.25  106507930.00  22962500.00  24120000.00   25.00     3.38   
max   3499.00 2323180070.00 706629000.00 563753000.00  285.00    22.08   

       주말방문비율    구매주기      성별  
count 3500.00 3500.00 3500.00  
mean     0.31   20.96    0.38  
std      0.29   24.75    0.48  
min      0.00    0.00    0.00  
25%      0.03    4.00    0.00  
50%      0.26   13.00    0.00  
75%      0.45   28.00    1.00  
max      1.00  166.00    1.00  


- 총 구매액과 최대 구매액에 마이너스 값 포함

    - 특정 도메인에 대한 지식 없는 상황에서 이상치 여부 판단 불가

        - X_test 데이터에도 마이너스 값이 있다면 이유 있는 데이터일 가능성이 높음

In [51]:
# test 확인
print(test.describe())

         회원ID          총구매액        최대구매액         환불금액    방문일수  방문당구매건수  \
count 2482.00       2482.00      2482.00       871.00 2482.00  2482.00   
mean  4740.50  101027523.71  21770481.06  25547158.99   19.52     2.82   
std    716.64  173213222.74  35049189.35  59440743.26   25.97     1.75   
min   3500.00  -37440000.00 -37440000.00     10000.00    1.00     1.00   
25%   4120.25    5076867.50   2884350.00   2414000.00    2.00     1.75   
50%   4740.50   30516865.00  10752500.00   8100000.00    9.00     2.43   
75%   5360.75  126425462.50  26277000.00  22280900.00   26.75     3.38   
max   5981.00 2861237540.00 593225000.00 871514400.00  222.00    15.88   

       주말방문비율    구매주기  
count 2482.00 2482.00  
mean     0.29   20.29  
std      0.28   24.11  
min      0.00    0.00  
25%      0.02    4.00  
50%      0.25   13.00  
75%      0.42   27.00  
max      1.00  177.00  


- test 데이터에서도 총 구매액과 최대 구매액 값에 마이너스 값이 존재하므로 x_train 데이터의 값이 이상치라고 판단하기엔 설득력이 약함

In [52]:
# train 데이터의 object 컬럼의 unique 개수 확인 : 주 구매상품 42개, 주 구매지점 24개
print(train.describe(include='O'))

# test 데이터에 있는 object 컬럼 확인 : 주 구매상품은 41개로 train 과 다름
print(test.describe(include='O'))

       주구매상품 주구매지점
count   3500  3500
unique    42    24
top       기타  본  점
freq     595  1077
       주구매상품 주구매지점
count   2482  2482
unique    41    24
top       기타  본  점
freq     465   726


- 주 구매상품의 종류 41개와 42개를 눈으로 비교하기에는 어려움이 있음

    - set() 함수를 사용하면 쉽게 차이가 있는 종류를 찾을 수 있음

        - set() 함수로 고유한 값(종류) 집합 형태로 변경

        - a - b : 집합 a에서 집합 b를 뺀 결과 출력

            - train 에는 있지만, test 에는 없는 '주 구매상품'의 고유한 값 출력

        - b - a : 집합 b에서 집합 a를 뺀 결과 출력

            - test 에는 있지만, train 에는 없는 '주 구매상품'의 고유한 값 출력

In [53]:
a = set(train['주구매상품'].unique())
b = set(test['주구매상품'].unique())

print(a-b)      # a 에는 소형가전이 포함되어있지만, b 에는 소형가전이 없음
print(b-a)

{'소형가전'}
set()


In [54]:
# train 결측치 확인 : 환불금액 2295 가 결측치로 확인됨
print(train.isnull().sum(), '\n')

# test 결측치 확인 : 환불금액 1611 이 결측치로 확인됨
print(test.isnull().sum())

회원ID          0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
방문일수          0
방문당구매건수       0
주말방문비율        0
구매주기          0
성별            0
dtype: int64 

회원ID          0
총구매액          0
최대구매액         0
환불금액       1611
주구매상품         0
주구매지점         0
방문일수          0
방문당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64


In [55]:
# gender 컬럼의 종류에 따른 개수 확인 : 클래스 0(여자) 2184개, 클래스 1(남자) 1316개
print(train['성별'].value_counts())

성별
0    2184
1    1316
Name: count, dtype: int64


- 필수 데이터 전처리 : 결측치 처리, object 형 컬럼 인코딩

    - 결측치가 있는 환불금액을 간단하게 0으로 대체

        - 환불금액이 없어서 결측치라고 가정

        - 결측치임을 표시하기 위해 특정 값(0)으로 변경한 것

In [56]:
# 데이터 전처리
train['환불금액'] = train['환불금액'].fillna(0)
test['환불금액'] = test['환불금액'].fillna(0)

# object 컬럼 레이블 인코딩
## 사이킷런의 LabelEncoder() 사용해 값을 숫자로 인코딩
## 여러 개의 컬럼이라면 반복문 사용
from sklearn.preprocessing import LabelEncoder
cols = ['주구매상품', '주구매지점']

for col in cols :
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
    
print(train.head(2))

   회원ID      총구매액     최대구매액       환불금액  주구매상품  주구매지점  방문일수  방문당구매건수  주말방문비율  \
0     0  68282840  11264000 6860000.00      5      0    19     3.89    0.53   
1     1   2136000   2136000  300000.00     21     19     2     1.50    0.00   

   구매주기  성별  
0    17   0  
1     1   0  


In [57]:
# train 데이터에서 타깃('성별') 데이터 분리
## pop() 함수에서 '성별'을 target 변수에 대입한 후 '성별' 제거
target = train.pop('성별')

# train 데이터를 활용해 검증 데이터(20%) 분할
# randomstate : 특정 숫자로 고정해야 매번 동일한 분할을 함
## 고정하지 않으면 코드를 실행할 때마다 분할 데이터가 달라져 성능 파악 어려움
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(2800, 10) (700, 10) (2800,) (700,)


In [61]:
# 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)
pred = model.predict_proba(X_val)
print(pred)

[[0.6  0.4 ]
 [0.8  0.2 ]
 [0.46 0.54]
 ...
 [0.5  0.5 ]
 [0.82 0.18]
 [0.47 0.53]]


- pred 변수에 있는 값 중 남자일 확률값만 필요

    - pred[행, 열]

        - pred[:, 0] : 여자 확률 전체 행 데이터

        - pred[:, 1] : 남자 확률 전체 행 데이터 (남자일 확률값만 선택)

- roc_auc : 1에 가까울수록 좋은 성능

In [62]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_val, pred[:,1]))

0.6150188336494536


In [63]:
# test 데이터로 머신러닝 모델 활용해 다시 예측
pred = model.predict_proba(test)
print(pred)

[[0.45 0.55]
 [0.71 0.29]
 [0.75 0.25]
 ...
 [0.44 0.56]
 [0.52 0.48]
 [0.81 0.19]]


In [64]:
# 제출 양식 형태로 데이터프레임 만들기
submit = pd.DataFrame({'pred' : pred[:, 1]})

print(submit.head(3))

   pred
0  0.55
1  0.29
2  0.25


In [66]:
# csv 파일로 저장 및 확인
submit.to_csv('result.csv', index=False)

result = pd.read_csv('result.csv')
print(result.head(3))

   pred
0  0.55
1  0.29
2  0.25


## 제 3유형

### ✏️ 타이타닉호에서 생존한 승객 및 사망한 승객의 정보를 포함한 데이터다. 이 데이터를 이용해 생존(Survived) 여부 예측하고자 한다. 각 문항의 답을 제출 형식에 맞춰 답안 작성 페이지에 입력하시오.

1. Gender 와 Survived 변수 간의 독립성 검정을 실시했을 때 카이제곱 통계량은? (반올림하여 소수 셋째 자리까지 계산)

2. Gender, SibSp, Parch, Fare 독립변수로 사용해 로지스틱 회귀 모델을 실시했을 때 Parch 변수의 계수 값은? (반올림하여 소수 셋째 자리까지 계산)

3. 2번 문제에서 추정된 로지스틱 회귀 모델에서 SibSp 변수가 한 단위 증가할 때 생존할 오즈비(Odds Ratio) 값은? (반올림하여 소수 셋째 자리까지 계산)

In [70]:
import pandas as pd

df = pd.read_csv('https://dataq.goorm.io/input-file/content?quizIndex=q_tgSCg_1698648409569&fileName=data%2FTitanic.csv')

print(df.head(), '\n')

print('#1')
crosstab = pd.crosstab(df['Gender'], df['Survived'])
print(crosstab)

from scipy.stats import chi2_contingency
print(chi2_contingency(crosstab))

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Gender   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male 22.00      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female 38.00      1   
2                             Heikkinen, Miss. Laina  female 26.00      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female 35.00      1   
4                           Allen, Mr. William Henry    male 35.00      0   

   Parch            Ticket  Fare Cabin Embarked  
0      0         A/5 21171  7.25   NaN        S  
1      0          PC 17599 71.28   C85        C  
2      0  STON/O2. 3101282  7.92   NaN        S  
3      0            113803 53.10  C123        S  
4      0            373450  8.05   NaN        S   

#1
Survived    0  

In [73]:
print('#2')
print(df[['Gender', 'SibSp', 'Parch', 'Fare']].head(3))

from statsmodels.formula.api import logit
X = df[['Gender', 'SibSp', 'Parch', 'Fare']].copy()
y = df['Survived']

# 2. Parch 변수의 계수
model = logit(formula='Survived ~ SibSp + Parch + Fare + Gender', data=df).fit()
print(model.summary())
print(round(-0.2007, 3))

#2
   Gender  SibSp  Parch  Fare
0    male      1      0  7.25
1  female      1      0 71.28
2  female      0      0  7.92
Optimization terminated successfully.
         Current function value: 0.482065
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      886
Method:                           MLE   Df Model:                            4
Date:                Fri, 22 Nov 2024   Pseudo R-squ.:                  0.2761
Time:                        16:48:31   Log-Likelihood:                -429.52
converged:                       True   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 1.192e-69
                     coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------

In [76]:
print('#3')
import numpy as np
odds_ratio = np.exp(model.params['SibSp'])
print(odds_ratio)
print(round(odds_ratio,3 ))

#3
0.7019508186162718
0.702
